In [6]:
!pip install tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [ ]:
!pip install openpyxl

In [1]:
!pip install sqlalchemy pymysql

  Using cached PyMySQL-1.1.1-py3-none-any.whl.metadata (4.4 kB)
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ------------------------- -------------- 1.3/2.1 MB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 9.0 MB/s eta 0:00:00
Using cached PyMySQL-1.1.1-py3-none-any.whl (44 kB)


In [2]:
!pip install watchdog

In [3]:
import os
import pandas as pd
from tabulate import tabulate
from sqlalchemy import create_engine
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Path folder yang dipantau
folder_path = 'C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel'

# Daftar sheet yang ingin diekstrak
sheets_to_extract = ['1210000', '1311000', '1510000']

# Fungsi untuk memproses dan mengekspor data ke MySQL
def process_and_export(file_path):
    # Dataframe gabungan
    combined_data = pd.DataFrame(columns=['keterangan', 'value', 'emiten', 'nama_perusahaan', 'kuartal', 'item'])

    try:
        # Membaca file Excel
        excel_file = pd.ExcelFile(file_path)

        # Ekstrak 'nama_perusahaan' dan 'kuartal' dari sheet '1000000'
        nama_perusahaan = None
        kuartal = None
        if '1000000' in excel_file.sheet_names:
            sheet_1000000 = excel_file.parse('1000000')
            
            # Memastikan file memiliki setidaknya 2 baris pertama untuk ekstraksi
            if len(sheet_1000000) >= 2:
                nama_perusahaan = sheet_1000000.iloc[4, 1]
                kuartal = sheet_1000000.iloc[19, 1]
                emiten = sheet_1000000.iloc[6, 1]   
                tahun  = sheet_1000000.iloc[20, 1]    

        # Loop untuk membaca sheet yang dipilih
        for sheet in sheets_to_extract:
            if sheet in excel_file.sheet_names: 
                print(f"Sheet '{sheet}' ditemukan dalam file {file_path}, memproses data...")
                data = excel_file.parse(sheet)

                # Drop 3 baris pertama
                data = data.iloc[3:]

                # Hanya mengambil 2 kolom pertama, jika ada
                data_selected = data.iloc[1:, :2]

                # Mengganti nama kolom menjadi 'keterangan' dan 'value'
                data_selected.columns = ['keterangan', 'value']

                # Menambahkan kolom 'Emiten' dengan nilai 'emiten'
                data_selected['emiten'] = emiten

                # Menambahkan kolom 'nama_perusahaan', 'kuartal', dan 'tahun
                data_selected['nama_perusahaan'] = nama_perusahaan
                data_selected['kuartal'] = kuartal
                data_selected['tahun'] = tahun
                # Mengambil posisi string pertama dengan memisahkan berdasarkan '-'
                data_selected['tahun'] = data_selected['tahun'].str.split('-').str[0]
                
                # Menambahkan kolom 'item'
                data_selected['item'] = data.iloc[0, 0] 
                data_selected['notes'] = '-'

                # Menggabungkan ke dataframe utama
                combined_data = pd.concat([combined_data, data_selected], ignore_index=True)


        # Menyesuaikan posisi kolom
        columns_order = ['emiten', 'nama_perusahaan', 'kuartal', 'tahun', 'item', 'keterangan', 'value', 'notes']
        combined_data = combined_data[columns_order]

        # Mengubah atau menyeleraskan data 'nama_perusahaan'
        combined_data['nama_perusahaan'] = (
            combined_data['nama_perusahaan']
            .str.upper() 
            .str.replace('TBK.', 'TBK', regex=True)
        )
        combined_data['nama_perusahaan'] = combined_data['nama_perusahaan'].apply(
            lambda x: f"PT {x}" if not x.startswith('PT ') else x
        )

        # Mengubah nama kolom sesuai database
        combined_data.columns = ['emiten', 'nama_perusahaan', 'kuartal', 'tahun', 'item', 'keterangan', 'value_calk', 'notes']
        # Drop kolom 'value_calk' dengan nilai NULL
        combined_data.dropna(subset=['value_calk'], inplace=True)

        # Pengaturan koneksi MySQL
        mysql_user = 'root' 
        mysql_password = 'Dika007!#'   
        mysql_host = 'localhost'  
        mysql_port = '3306' 
        db_name = 'db_calk'  

        # Membuat koneksi ke MySQL menggunakan SQLAlchemy
        engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{db_name}')

        # Mengekspor dataframe ke Database MySQL
        combined_data.to_sql('tb_calk', con=engine, if_exists='append', index=False)

        print(f"\nData dari {file_path} telah berhasil disisipkan ke tabel 'tb_calk' dalam database '{db_name}'.")

        # Menandai file sudah diproses
        processed_file_path = file_path + '.processed'
        with open(processed_file_path, 'w') as f:
            f.write('Processed')

    except Exception as e:
        print(f"Error saat memproses file {file_path}: {e}")


# Class untuk menangani event file baru
class MyHandler(FileSystemEventHandler):
    def on_created(self, event):
        # Skip folder
        if event.is_directory:
            return
        # Mencari file pada folder dengan .xlsx atau .xls
        if event.src_path.endswith(('.xlsx', '.xls')): 
            processed_file = event.src_path + '.processed'
            if not os.path.exists(processed_file):  
                print(f"\nFile baru ditemukan: {event.src_path}")
                process_and_export(event.src_path)
            else:
                print(f"File {event.src_path} sudah diproses sebelumnya, melewatkan...")

# Fungsi untuk memproses semua file yang sudah ada
def process_existing_files():
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith(('.xlsx', '.xls')):  
            processed_file = file_path + '.processed'
            if not os.path.exists(processed_file):  
                print(f"\nMemproses file yang sudah ada: {file_path}")
                process_and_export(file_path)
            else:
                print(f"File {file_path} sudah diproses sebelumnya, melewatkan...")

# Membuat observer untuk memantau folder
observer = Observer()
event_handler = MyHandler()
observer.schedule(event_handler, folder_path, recursive=False)

# Memproses file yang sudah ada di folder
process_existing_files()

# Memantau folder
observer.start()
print(f"Memantau folder: {folder_path} untuk file baru...")

# Program terus berjalan sampai dihentikan secara manual
try:
    while True:
        pass 
except KeyboardInterrupt:
    observer.stop()
    print("\nPemantauan dihentikan.")
observer.join()



Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx telah berhasil disisipkan ke tabel 'tb_calk' dalam database 'db_calk'.

Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx telah berhasil disisipkan ke tabel 'tb_calk' dalam database 'db_calk'.

Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx


...

Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx telah berhasil disisipkan ke tabel 'tb_calk' dalam database 'db_calk'.

Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-II-AGAR.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-II-AGAR.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-II-AGAR.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-II-AGAR.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-II-AGAR.xlsx telah berhasil disisipkan ke tabel 'tb_calk' dalam database 'db_calk'.
Memantau folder: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel untuk file baru...

File baru ditemukan: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABMM.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABMM.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABMM.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABMM.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABMM.xlsx telah berhasil disisipkan ke tabel 'tb_calk' dalam database 'db_calk'.

Pemantauan dihentikan.


In [7]:
import os
import pandas as pd
from tabulate import tabulate

# Path folder yang dipantau
folder_path = 'C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel'

# Daftar sheet yang ingin diekstrak
sheets_to_extract = ['1210000', '1311000', '1510000']

# Fungsi untuk memproses dan menampilkan hasil ekstraksi
def process_and_display(file_path):
    # Dataframe gabungan
    combined_data = pd.DataFrame(columns=['keterangan', 'value', 'emiten', 'nama_perusahaan', 'kuartal', 'item'])

    try:
        # Membaca file Excel
        excel_file = pd.ExcelFile(file_path)

        # Ekstrak 'nama_perusahaan' dan 'kuartal' dari sheet '1000000'
        nama_perusahaan = None
        kuartal = None
        if '1000000' in excel_file.sheet_names:
            sheet_1000000 = excel_file.parse('1000000')
            
            # Memastikan file memiliki setidaknya 2 baris pertama untuk ekstraksi
            if len(sheet_1000000) >= 2:
                nama_perusahaan = sheet_1000000.iloc[4, 1]
                kuartal = sheet_1000000.iloc[19, 1]
                emiten = sheet_1000000.iloc[6, 1]   
                tahun  = sheet_1000000.iloc[20, 1]    

        # Loop untuk membaca sheet yang dipilih
        for sheet in sheets_to_extract:
            if sheet in excel_file.sheet_names: 
                print(f"Sheet '{sheet}' ditemukan dalam file {file_path}, memproses data...")
                data = excel_file.parse(sheet)

                # Drop 3 baris pertama
                data = data.iloc[3:]

                # Hanya mengambil 2 kolom pertama, jika ada
                data_selected = data.iloc[1:, :2]

                # Mengganti nama kolom menjadi 'keterangan' dan 'value'
                data_selected.columns = ['keterangan', 'value']

                # Menambahkan kolom 'Emiten' dengan nilai 'emiten'
                data_selected['emiten'] = emiten

                # Menambahkan kolom 'nama_perusahaan', 'kuartal', dan 'tahun
                data_selected['nama_perusahaan'] = nama_perusahaan
                data_selected['kuartal'] = kuartal
                data_selected['tahun'] = tahun
                # Mengambil posisi string pertama dengan memisahkan berdasarkan '-'
                data_selected['tahun'] = data_selected['tahun'].str.split('-').str[0]
                
                # Menambahkan kolom 'item'
                data_selected['item'] = data.iloc[0, 0] 
                data_selected['notes'] = '-'

                # Menggabungkan ke dataframe utama
                combined_data = pd.concat([combined_data, data_selected], ignore_index=True)

        # Menyesuaikan posisi kolom
        columns_order = ['emiten', 'nama_perusahaan', 'kuartal', 'tahun', 'item', 'keterangan', 'value', 'notes']
        combined_data = combined_data[columns_order]

        # Mengubah atau menyeleraskan data 'nama_perusahaan'
        combined_data['nama_perusahaan'] = (
            combined_data['nama_perusahaan']
            .str.upper() 
            .str.replace('TBK.', 'TBK', regex=True)
        )
        combined_data['nama_perusahaan'] = combined_data['nama_perusahaan'].apply(
            lambda x: f"PT {x}" if not x.startswith('PT ') else x
        )

        # Mengubah nama kolom sesuai kebutuhan
        combined_data.columns = ['emiten', 'nama_perusahaan', 'kuartal', 'tahun', 'item', 'keterangan', 'value_calk', 'notes']
        # Drop kolom 'value_calk' dengan nilai NULL
        combined_data.dropna(subset=['value_calk'], inplace=True)

        print(f"\nHasil ekstraksi dari file {file_path}:")
        print(tabulate(combined_data, headers='keys', tablefmt='grid'))

    except Exception as e:
        print(f"Error saat memproses file {file_path}: {e}")

# Fungsi untuk memproses semua file yang ada di folder
def process_existing_files():
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith(('.xlsx', '.xls')):  
            print(f"\nMemproses file: {file_path}")
            process_and_display(file_path)

# Memproses file yang ada di folder
process_existing_files()



Memproses file: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...

Hasil ekstraksi dari file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx:
+-----+----------+---------------------+---------------------------+---------+-----------------------------------------------------+--------------------------------------------------------------------------------------------------+--------------+---------+
|     | emiten   | nama_perusahaan     | kuartal                   |   tahun | item                                                | keterangan                                                                

c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABMM.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABMM.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABMM.xlsx, memproses data...

Hasil ekstraksi dari file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABMM.xlsx:
+-----+----------+----------------------+---------------------------+---------+-----------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------+-------------------+---------+
|     | emiten   | nama_perusahaan      | kuartal                   |   tahun | item                                                | keterangan                               

...

c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...

Hasil ekstraksi dari file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx:
+-----+----------+---------------------------+---------------------------+---------+-----------------------------------------------------+--------------------------------------------------------------------------------------------------------------+------------------+---------+
|     | emiten   | nama_perusahaan           | kuartal                   |   tahun | item                                                | keterangan                                    

c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-II-AGAR.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-II-AGAR.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-II-AGAR.xlsx, memproses data...

Hasil ekstraksi dari file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-II-AGAR.xlsx:
+-----+----------+----------------------------+-----------------------------+---------+-----------------------------------------------------+--------------------------------------------------------------------------------------------------------------+------------------+---------+
|     | emiten   | nama_perusahaan            | kuartal                     |   tahun | item                                                | keterangan                          

c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
